### Assignment 1: Introduction ###

COURSERA CAPSTONE PROJECT

This notebook will be mainly used for the capstone project.

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Assignment 2: Segmenting and Clustering Neighborhoods in Toronto ###

In [3]:
import numpy as np
import pandas as pd

Scraping table from wikipedia link and deleting rows for which Borough is not assigned.

In [4]:
url = r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url) # Returns list of all tables on page
df = tables[0] # Select table of interest

df=df[~df.Borough.str.contains("Not assigned")] #deleting not assigned borough rows

Question 1

Grouping neighborhood elements having same postcode.

In [5]:
df=df.groupby('Postcode').agg(lambda x: ', '.join(set(x))).reset_index()
df.set_index("Postcode",inplace=True)
df

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
M1E,Scarborough,"Guildwood, West Hill, Morningside"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Kennedy Park, East Birchmount Park, Ionview"
M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside"


In [32]:
df.shape

(103, 3)

Question 2

Adding to the dataframe location (latitude,longitude) informations.

In [2]:
import pandas as pd
dfl=pd.read_csv("https://cocl.us/Geospatial_data")
dfl.set_index("Postal Code",inplace=True)

In [6]:
dff=pd.concat([df, dfl], axis=1)
dff.reset_index(inplace=True)
dff=dff.rename({'index': 'Postal Code'}, axis=1)
dff.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Question 3

Clustering Neighborhoods in Toronto.

In [7]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [8]:
latitude = 43.6529
longitude = -79.3849
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))# create map of Toronto using latitude and longitude values

map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dff['Latitude'], dff['Longitude'], dff['Borough'], dff['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

The geograpical coordinate of Toronto are 43.6529, -79.3849.


Choosing for the analysis all the boroughs containing word Toronto in their name.

In [9]:
dffc = pd.DataFrame(dff[dff['Borough'].str.contains("Toronto")])
dffc

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Rathnelly, South Hill, Forest Hill SE, Summerh...",43.686412,-79.400049


In [13]:
CLIENT_ID = 'IFUAXALYJK3F4XL3T3M2SLCYDXJVDFZKNRR4XU5FG22WTFJG' # your Foursquare ID
CLIENT_SECRET = 'HAXDDXAB5GDBBWLO4KADTVNAY3WEMTIS5PTTQDPLEYH0SL4Z' # your Foursquare Secret
VERSION = '20200104' # Foursquare API version
radius=500
LIMIT=100


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IFUAXALYJK3F4XL3T3M2SLCYDXJVDFZKNRR4XU5FG22WTFJG
CLIENT_SECRET:HAXDDXAB5GDBBWLO4KADTVNAY3WEMTIS5PTTQDPLEYH0SL4Z


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
# type your answer here

tor_venues = getNearbyVenues(names=dffc['Neighborhood'],
                                   latitudes=dffc['Latitude'],
                                   longitudes=dffc['Longitude']
                                  )
tor_venues.head()

The Beaches
Riverdale, The Danforth West
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Rathnelly, South Hill, Forest Hill SE, Summerhill West, Deer Park
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Toronto Islands, Union Station, Harbourfront East
Toronto Dominion Centre, Design Exchange
Victoria Hotel, Commerce Court
Roselawn
Forest Hill North, Forest Hill West
Yorkville, North Midtown, The Annex
Harbord, University of Toronto
Kensington Market, Grange Park, Chinatown
Island airport, CN Tower, Bathurst Quay, South Niagara, Harbourfront West, King and Spadina, Railway Lands
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Parkdale Village, Brockton, Exhibition Place
High Park, The Junction Sout

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Counting number of venues per each neighborhood.

In [16]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,82,82,82,82,82,82
Christie,17,17,17,17,17,17
Church and Wellesley,85,85,85,85,85,85
Davisville,30,30,30,30,30,30
Davisville North,7,7,7,7,7,7
"Dufferin, Dovercourt Village",18,18,18,18,18,18


Analysis is carried out via one hot encoding of the neighborhoods and finding the 10 most common per each category.

In [17]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

#tor_onehot.head()

tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Central Bay Street,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,0.00,...,0.000000,0.000000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.00
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Church and Wellesley,0.023529,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.011765,0.000000,0.011765,0.011765,0.00
6,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.00
9,"Forest Hill North, Forest Hill West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Creperie
1,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Auto Workshop,Skate Park,Smoke Shop,Burrito Place,Spa,Farmers Market,Fast Food Restaurant,Butcher,Restaurant
2,"Cabbagetown, St. James Town",Coffee Shop,Café,Pub,Italian Restaurant,Pizza Place,Bakery,Restaurant,Japanese Restaurant,Breakfast Spot,Caribbean Restaurant
3,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Sandwich Place,Burger Joint,Gym / Fitness Center,Juice Bar,Japanese Restaurant,Salad Place
4,Christie,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Candy Store,Restaurant,Diner,Athletics & Sports,Baby Store


Clustering

In [38]:
# set number of clusters
kclusters = 10

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 4, 0, 4, 0, 4, 5, 4, 7], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = dffc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged # check the last columns!

tor_grouped

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,9,Pub,Other Great Outdoors,Coffee Shop,Health Food Store,Trail,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Furniture / Home Store,Yoga Studio,Indian Restaurant,Pub,Diner
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,4,Sandwich Place,Park,Pub,Burger Joint,Burrito Place,Liquor Store,Italian Restaurant,Fast Food Restaurant,Fish & Chips Shop,Steakhouse
43,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Gastropub,Gym / Fitness Center,Fish Market,Pizza Place,Pet Store
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Dim Sum Restaurant,Concert Hall,Convenience Store,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,5,Gym,Clothing Store,Breakfast Spot,Food & Drink Shop,Sandwich Place,Hotel,Park,Creperie,Coworking Space,Comic Shop
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Gym / Fitness Center,Mexican Restaurant,Salon / Barbershop,Restaurant,Rental Car Location,Pet Store,Park
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,4,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Gym,Coffee Shop,Café,Sushi Restaurant,Park,Brewery
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,6,Gym,Playground,Trail,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
49,M4V,Central Toronto,"Rathnelly, South Hill, Forest Hill SE, Summerh...",43.686412,-79.400049,0,Coffee Shop,Pub,Pizza Place,Light Rail Station,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Fried Chicken Joint


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters